In [1]:
import sys
sys.path.insert(0, '..')

import yaml
import numpy as np
import jax.numpy as jnp
import pandas as pd
from jax import vmap
import jax
import matplotlib.pyplot as plt

# Creating a combined config file:

Loading files:

In [2]:
# Position of hexagon pixels in flashcam:
flashpix = np.load('flashpix.npy')
# Position of hexagon pixels in hess2 era
twopix = np.load('twopix.npy')

In [3]:
# Positions of mirrors:
df = pd.read_csv('hessII-mirrors-0876-installed.dat', comment='#', sep='\s+')
df = df.apply(lambda x: x.astype(str).str.replace(',', ''))
df = df.astype(float)

mirrors_xyd = np.array(df[['y', 'x', 'd']].values)/100

In [4]:
# Positions of shadowing structure:
df = pd.read_csv('hess2_shadow.dat', comment='#', sep='\s+')
df = df.apply(lambda x: x.astype(str).str.replace(',', ''))
df = df.astype(float)

cyl_points1 = np.array(df[['x1', 'y1', 'z1']].values)*0.01
cyl_points2 = np.array(df[['x2', 'y2', 'z2']].values)*0.01
cyl_radius = np.array(df[['diam.']].values/2)/100

# Some useful code:

In [5]:
def spherical_surface(radius, points):
    Z = radius-np.sqrt(radius**2-(points[...,0]**2+points[...,1]**2))
    N = np.stack([-points[...,0], -points[...,1], radius - Z], axis=-1)
    return np.stack([points[...,0], points[...,1], Z], axis=-1), N/radius


def parabolic_surface(focal_length, points):
    a = 1.0 / (4.0 * focal_length)
    Z = a * (points[..., 0]**2 + points[..., 1]**2)
    ones = jnp.ones_like(Z)
    N = jnp.stack([-2*a*points[..., 0], -2*a*points[..., 1], ones], axis=-1)
    norm = jnp.sqrt(4*a*Z + 1)
    positions = jnp.stack([points[..., 0], points[..., 1], Z], axis=-1)

    return positions, N / norm[..., None]


def look_at_euler(mirror_pos, target_pos=jnp.array([0., 0., 0.]), up=jnp.array([0., 1., 0.])):
    """
    Compute euler angles to look from mirror_pos towards target_pos.

    Args:
        mirror_pos: Position to look from (3,)
        target_pos: Position to look at (3,)
        up: Up vector (3,)

    Returns:
        Euler angles (3, )
    """
    forward = target_pos - mirror_pos
    forward = forward / jnp.linalg.norm(forward)

    right = jnp.cross(up, forward)
    right = right / jnp.linalg.norm(right)

    up_corrected = jnp.cross(forward, right)

    # Rotation matrix: local -> world
    R = jnp.column_stack([right, up_corrected, forward])

    # Compute tilt
    sy = -R[2,0]  # sin(tilt)
    cy = jnp.sqrt(1 - sy**2)

    # Avoid gimbal problem
    tip = jax.lax.cond(
        cy > 1e-6,
        lambda _: jnp.arctan2(R[2,1], R[2,2]),
        lambda _: 0.0,
        operand=None
    )
    rotation = jax.lax.cond(
        cy > 1e-6,
        lambda _: jnp.arctan2(R[1,0], R[0,0]),
        lambda _: jnp.arctan2(-R[0,1], R[1,1]),
        operand=None
    )
    tilt = jnp.arcsin(sy)

    return jnp.rad2deg(jnp.array([tip, tilt, rotation]))

# Using ConfigBuilder:

In [6]:
from builder import TelescopeConfigBuilder

In [9]:
builder = TelescopeConfigBuilder('CT5')

# Flashcam sensor
builder.add_hexagon_sensor_array(
    sensor_id='flashcam',
    position=[0, 0, 36.25],
    orientation=[0, 180, 0],
    pixel_x = flashpix[:,0],
    pixel_y = flashpix[:,1])

# Old HESS2 camera
builder.add_hexagon_sensor_array(
    sensor_id='hess2_cam',
    position=[0, 0, 36.25],
    orientation=[0, 180, 0],
    pixel_x = twopix[:,0],
    pixel_y = twopix[:,1])

builder.add_square_sensor_array(
    sensor_id='camera_lid',
    position=[0, 0, 36.0],
    orientation=[0, 180, 0],
    width = 1000,
    height = 1000,
    bounds = [-1.00,1.00,-1.00,1.00])

# Mirror array

## Add spherical template:
builder.add_mirror_template('spherical_72m', curvature=1/(36.4*2), conic=0.0, aspheric_coeffs=[])

## Calculate intersection point for each mirror point
f = 36
z_val = 2*f + np.sum(mirrors_xyd[:,:2]**2, axis=1) / (4*f)
p_orient = jnp.stack([jnp.zeros_like(z_val), jnp.zeros_like(z_val), z_val]).T

## For all mirrors, calculate position and rotation:
Tmirrors, _ = parabolic_surface(36, mirrors_xyd[:,:2])
Rmirrors = np.array(vmap(look_at_euler, in_axes= (0, 0))(Tmirrors, p_orient))
Rmirrors[:,2] = 0

## Define hexagon vertices
D = 0.9  # flat-to-flat distance in meters
R = D / np.sqrt(3)  # distance from center to vertex

# angles for vertices
angles = np.deg2rad(np.arange(0, 360, 60))

# coordinates of vertices
vertices = list(zip((R * np.sin(angles)).tolist(),(R * np.cos(angles)).tolist()))

## Add mirrors one by one
for i in range(len(Tmirrors)):
    builder.add_mirror_polygon(
        mirror_id='M_{}'.format(i),
        template='spherical_72m',
        position=Tmirrors[i],
        orientation=Rmirrors[i],
        vertices=vertices
        )
    
# Add shadowing obstructions:
for i in range(len(cyl_points1)):
    if i<64:
        cyl_string = 'Main_Mast_{}'.format(i)
    if i>=64 and i < 220:
        cyl_string = 'Truss_{}'.format(i-64)
    if i>=220 and i < 264:
        cyl_string = 'Camera_Mount_{}'.format(i-220)
    if i>=264 and i < 270:
        cyl_string = 'Camera_Support_{}'.format(i-264)
    if i==270:
        cyl_string = 'Camera_Body'
        
    builder.add_obstruction_cylinder(cyl_string,
                                     cyl_points1[i],
                                     cyl_points2[i],
                                     cyl_radius[i])
    
builder.save('CT5.yaml')

Saved telescope config to CT5.yaml
